In [1]:
import ipytest
import pytest

ipytest.autoconfig()

In [2]:
from pathlib import Path

In [3]:
data_path = Path.cwd() / 'data' / 'day4_input.txt'

In [4]:
data_path.exists()

True

#### Part 1

In [5]:
import numpy as np
import re

from collections.abc import Iterable
from itertools import groupby

In [6]:
def parse_grouped_strings_to_int_lists(grouped_string: Iterable[str]) -> list[int]:
    return [list(map(int, re.split(r'[,\s]+', line.strip()))) for line in grouped_string]
        

def read_arrays_from_file(file_path: Path) -> list[np.ndarray]:
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    arrays = [
        np.array(parse_grouped_strings_to_int_lists(group))
        for is_non_empty, group in groupby(lines, key=lambda x: bool(x.strip()))
        if is_non_empty
    ]
    
    return arrays

In [7]:
%%ipytest -vv

def test_parse_grouped_strings_to_int_lists():
    assert parse_grouped_strings_to_int_lists(['1, 2, 3\n', '4 5 6\n']) == [[1, 2, 3], [4, 5, 6]]
    assert parse_grouped_strings_to_int_lists(['   1, 2, 3  \n  ', '4 5 6\n']) == [[1, 2, 3], [4, 5, 6]]


======================================= test session starts ========================================
platform darwin -- Python 3.12.8, pytest-8.3.5, pluggy-1.5.0 -- /Users/hariravindran/Documents/workstation/Advent-of-Code-2021/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/hariravindran/Documents/workstation/Advent-of-Code-2021
configfile: pyproject.toml
collecting ... collected 1 item

t_b0c4c23717e848ad9723a03972274c5c.py::test_parse_grouped_strings_to_int_lists PASSED        [100%]

======================================== 1 passed in 0.01s =========================================


In [8]:
%%ipytest -vv

def test_read_arrays_from_file(tmp_path):
    DATA_TEXT = """
    46,12,57,37,14,78,31,71,87,52,64,97,10,35,54,36

    37 72 60 35 89
    32 49  4 77 82
    """

    test_data_path = tmp_path / 'test_data.txt'
    test_data_path.write_text(DATA_TEXT)

    bingo_list_of_numbers, *bingo_boards = read_arrays_from_file(test_data_path)
    
    assert np.array_equal(bingo_list_of_numbers, np.array([[46, 12, 57, 37, 14, 78, 31, 71, 87, 52, 64, 97, 10, 35, 54, 36]]))
    assert np.array_equal(bingo_boards, [np.array([[37, 72, 60, 35, 89], [32, 49, 4, 77, 82]])])

======================================= test session starts ========================================
platform darwin -- Python 3.12.8, pytest-8.3.5, pluggy-1.5.0 -- /Users/hariravindran/Documents/workstation/Advent-of-Code-2021/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/hariravindran/Documents/workstation/Advent-of-Code-2021
configfile: pyproject.toml
collecting ... collected 1 item

t_b0c4c23717e848ad9723a03972274c5c.py::test_read_arrays_from_file 

PASSED                     [100%]

======================================== 1 passed in 0.01s =========================================


In [9]:
bingo_array_of_numbers, *bingo_boards = read_arrays_from_file(data_path)

In [10]:
bingo_list_of_numbers = bingo_array_of_numbers.flatten().tolist()

In [11]:
bingo_boards[1]

array([[41, 94, 77, 43, 87],
       [ 2, 17, 82, 96, 25],
       [95, 49, 32, 12,  9],
       [59, 33, 67, 71, 64],
       [88, 54, 93, 85, 30]])

In [12]:
np.sum(bingo_boards[1][~np.isin(bingo_boards[1], [41, 94, 77, 43, 87])])

np.int64(1063)

In [69]:
def check_win(mask: np.ndarray) -> bool:
    rows_win = np.sum(mask.all(axis=1))  # Count rows where all elements are True
    cols_win = np.sum(mask.all(axis=0))  # Count columns where all elements are True
    return (rows_win + cols_win) == 1

def calculate_score(winning_board: np.ndarray, drawn_numbers: list[int]) -> int:
    return np.sum(winning_board[~np.isin(winning_board, drawn_numbers)]) * drawn_numbers[-1]

#### Part 1

In [70]:
sublists_of_numbers = (
    bingo_list_of_numbers[:i] for i in range (1, len(bingo_list_of_numbers)+1)
)

for i in range(1, len(bingo_list_of_numbers) + 1):
    list_of_drawn_numbers = bingo_list_of_numbers[:i]
    winning_board = next((board for board in bingo_boards if check_win(np.isin(board, list_of_drawn_numbers))), None)
    if winning_board is not None:
        winning_score = calculate_score(winning_board=winning_board, drawn_numbers=list_of_drawn_numbers)
        print(winning_score)
        break

74320


#### Part 2

In [71]:
sublists_of_numbers = (
    bingo_list_of_numbers[:i] for i in range (1, len(bingo_list_of_numbers)+1)
)

winning_boards_and_numbers: list[tuple[np.ndarray, list[int]]] = []
for i in range(1, len(bingo_list_of_numbers) + 1):
    list_of_drawn_numbers = bingo_list_of_numbers[:i]
    winning_board = next((board for board in bingo_boards if check_win(np.isin(board, list_of_drawn_numbers))), None)
    if winning_board is not None:
        winning_boards_and_numbers.append((winning_board, list_of_drawn_numbers))

In [72]:
len(winning_boards_and_numbers)

71

In [76]:
calculate_score(winning_board=winning_boards_and_numbers[-2][0], drawn_numbers=winning_boards_and_numbers[-2][1])

np.int64(17884)

In [77]:
np.isin(winning_boards_and_numbers[-2][0], winning_boards_and_numbers[-2][1])

array([[ True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True],
       [ True, False, False,  True,  True],
       [ True,  True,  True, False,  True],
       [ True, False,  True, False, False]])